In [ ]:
import os
import sys
sys.path.append("../")

from tqdm.notebook import tqdm
import numpy as np
from preproc.featureExtractor import get_ecg_features, subject_extractor, get_ecg_features_np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
BASE = '/NFS/Users/moonsh/data/mesa/preproc/'
data_dir = os.path.join(BASE, 'npy')
save_dir = os.path.join(BASE, 'features')

subject_ids = subject_extractor(data_dir)

In [ ]:
error = []
for subj_id in tqdm(subject_ids):
    try:
        get_ecg_features_np(subj_id, data_dir, save_dir)
    except:
        print(f"Error in Subject {subj_id}")
        error.append(subj_id)